In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

while any(marker in os.getcwd() for marker in ('exercises', 'notebooks', 'students', 'research', 'projects')):
    os.chdir("..")
sys.path.append('src')

In [2]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mlflow
import optuna
import optuna.visualization as vis
from dotenv import load_dotenv
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_log_error # Metric used in the competition for evaluation

/Users/wojciechjurewicz/anaconda3/envs/mda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Set MLflow tracking URI to local directory
mlflow.set_tracking_uri("projects/proj_2_team_4/mlruns")

# Use for running ui:
# mlflow ui --backend-store-uri "Absolute path"
# e.g.: mlflow ui --backend-store-uri "/Users/wojciechjurewicz/Desktop/Multivariate Data Analysis/Lab/mda2425/projects/proj_2_team_4/mlruns"



In [4]:
# Load environment variables from the correct .env file location
env_path = 'projects/proj_2_team_4/.env'
load_dotenv(env_path)

True

In [5]:
# Get dataset path from environment variable
train_preprocessed_path = os.getenv('TRAIN_PREPROCESSED_PATH')
test_preprocessed_path = os.getenv('VALID_PREPROCESSED_PATH')

df_train = pd.read_csv(train_preprocessed_path)
df_test = pd.read_csv(test_preprocessed_path)

In [6]:
train_df, valid_df = train_test_split(df_train, test_size=0.10, random_state=42)

In [7]:
X_train = train_df.drop(columns=["SalePrice"])
y_train = train_df["SalePrice"]

X_valid = valid_df.drop(columns=["SalePrice"])
y_valid = valid_df["SalePrice"]

X_test = df_test.drop(columns=["SalePrice"])
y_test = df_test["SalePrice"]

In [8]:

y_train.shape


(278174,)

In [9]:
learning_rate = 0.2
max_depth = 8
n_estimators = 1000
subsample = 0.6
tree_method = 'approx'

In [10]:
pipeline = Pipeline([
    ('xgb', XGBRegressor(
        learning_rate=learning_rate, 
        max_depth=max_depth, 
        n_estimators=n_estimators, 
        subsample=subsample, 
        tree_method=tree_method
    ))
])


In [11]:
# # override Optuna's default logging to ERROR only
# optuna.logging.set_verbosity(optuna.logging.ERROR)

# # define a logging callback that will report on only new challenger parameter configurations if a
# # trial has usurped the state of 'best conditions'


# def champion_callback(study, frozen_trial):
#   """
#   Logging callback that will report when a new trial iteration improves upon existing
#   best trial values.

#   Note: This callback is not intended for use in distributed computing systems such as Spark
#   or Ray due to the micro-batch iterative implementation for distributing trials to a cluster's
#   workers or agents.
#   The race conditions with file system state management for distributed trials will render
#   inconsistent values with this callback.
#   """

#   winner = study.user_attrs.get("winner", None)

#   if study.best_value and winner != study.best_value:
#       study.set_user_attr("winner", study.best_value)
#       if winner:
#           improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
#           print(
#               f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
#               f"{improvement_percent: .4f}% improvement"
#           )
#       else:
#           print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

In [12]:
mlflow.set_experiment("XGBoost_Pareto_Front")

def objective(trial):
    start = time.time()

    # Hyperparameter search space
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "n_estimators": trial.suggest_int("n_estimators", 100, 2200),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "tree_method": trial.suggest_categorical("tree_method", ["approx", "hist"]),
    }

    # Start MLflow run (one per trial)
    with mlflow.start_run(nested=True):
        mlflow.log_params(params)  # Save all hyperparameters

        # Optional tags
        mlflow.set_tags({
            "trial_number": trial.number,
            "optimizer_engine": "optuna",
            "model_family": "xgboost",
            "phase": "search"
        })

        # Train model
        pipeline = Pipeline([
            ('xgb', XGBRegressor(**params, n_jobs=-1))
        ])
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_valid)
        y_pred = np.maximum(0, y_pred)

        # Calculate objective values
        training_time = time.time() - start
        rmsle = root_mean_squared_log_error(y_valid, y_pred)

        # Log metrics
        mlflow.log_metrics({
            "Validation RMSLE": rmsle,
            "training_time": training_time
        })

    return rmsle, training_time


2025/05/26 23:31:17 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost_Pareto_Front' does not exist. Creating a new experiment.


In [13]:
# Create study
study = optuna.create_study(directions=["minimize", "minimize"])
study.optimize(objective, n_trials=300, timeout=1800, show_progress_bar=True) # set n_trials to higher when doing final training and summary

# For multi-objective, get the list of best trials on the Pareto front
best_trials = study.best_trials

# Extract RMSLE and training_time from best trials
rmsle_values = [t.values[0] for t in best_trials]
time_values = [t.values[1] for t in best_trials]

# Normalize function to scale list to [0,1]
def normalize(lst):
    min_val, max_val = min(lst), max(lst)
    if max_val == min_val:
        return [0.0 for _ in lst]  # avoid division by zero if all equal
    return [(x - min_val) / (max_val - min_val) for x in lst]

rmsle_norm = normalize(rmsle_values)
time_norm = normalize(time_values)

alpha = 0.7  # weight for RMSLE vs training time

# Compute combined score for each trial
scores = [alpha * r + (1 - alpha) * t for r, t in zip(rmsle_norm, time_norm)]

# Get the trial with the lowest combined score
best_index = scores.index(min(scores))
best_trial = best_trials[best_index]
best_params = best_trial.params

# Train final model with best parameters
experiment = mlflow.set_experiment("Bluebook_for_bulldozers_XGBoost")
with mlflow.start_run(experiment_id=experiment.experiment_id):
    mlflow.log_params(best_params)

# Log tags
    mlflow.set_tags(
        tags={
            "project": "Bluebook for Bulldozers",
            "optimizer_engine": "optuna",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )
    
    # Train final model
    final_pipeline = Pipeline([
        ('xgb', XGBRegressor(**best_params))
    ])
    final_pipeline.fit(X_train, y_train)
    y_pred = final_pipeline.predict(X_test)

    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    rmsle = root_mean_squared_log_error(y_test, y_pred)

    # Log metrics
    mlflow.log_metrics({
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R2": r2,
        "RMSLE": rmsle
    })

    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"R-squared (Coefficient of Determination): {r2}")
    print(f"Root Mean Squared Log Error (RMSLE): {rmsle} - Metric used in competition")
    mlflow.sklearn.log_model(final_pipeline, "model")

    # Create and log visualization
    plt.scatter(y_test, y_pred, alpha=0.7)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', linewidth=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title('Actual vs Predicted')
    plt.grid(True)
    plt.axis('equal')
    plt.tight_layout()
    actual_vs_predicted_path = "actual_vs_predicted.png"
    plt.savefig(actual_vs_predicted_path)
    plt.close()

    mlflow.log_artifact(actual_vs_predicted_path)
    os.remove(actual_vs_predicted_path)

[I 2025-05-26 23:31:17,585] A new study created in memory with name: no-name-88eff0f3-e711-48f6-8f05-591dac2e460f
  0%|          | 1/300 [00:22<1:52:10, 22.51s/it, 22.51/1800 seconds]

[I 2025-05-26 23:31:40,116] Trial 0 finished with values: [0.25045686960220337, 22.50075626373291] and parameters: {'learning_rate': 0.13732680473655495, 'max_depth': 3, 'n_estimators': 1639, 'subsample': 0.5738291002648241, 'tree_method': 'hist'}.


  1%|          | 2/300 [00:43<1:48:09, 21.78s/it, 43.77/1800 seconds]

[I 2025-05-26 23:32:01,378] Trial 1 finished with values: [0.21005627512931824, 21.25073528289795] and parameters: {'learning_rate': 0.023208347581707246, 'max_depth': 9, 'n_estimators': 415, 'subsample': 0.7658909751428908, 'tree_method': 'approx'}.


  1%|          | 3/300 [00:53<1:19:47, 16.12s/it, 53.16/1800 seconds]

...

 20%|██        | 61/300 [29:34<55:55, 14.04s/it, 1774.95/1800 seconds]  

[I 2025-05-27 00:00:52,555] Trial 60 finished with values: [0.19246293604373932, 8.57093596458435] and parameters: {'learning_rate': 0.12284641402816608, 'max_depth': 12, 'n_estimators': 265, 'subsample': 0.7046377101997628, 'tree_method': 'hist'}.


 21%|██        | 62/300 [30:00<1:55:12, 29.04s/it, 1800.65/1800 seconds]


[I 2025-05-27 00:01:18,255] Trial 61 finished with values: [0.24867834150791168, 25.69218897819519] and parameters: {'learning_rate': 0.19132571928190817, 'max_depth': 3, 'n_estimators': 1285, 'subsample': 0.7296916537828325, 'tree_method': 'approx'}.
Mean Absolute Error (MAE): 6011.3967715370145
Mean Squared Error (MSE): 86203927.48737396
Root Mean Squared Error (RMSE): 9284.607018467392
R-squared (Coefficient of Determination): 0.8621614432988232
Root Mean Squared Log Error (RMSLE): 0.24227875279852001 - Metric used in competition


2025/05/27 00:01:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [14]:
vis.plot_pareto_front(study, target_names=["RMSLE", "Training Time"]).show()
